In [ ]:
import matplotlib.pylab as plt
import numpy as np

# Непараметрическая постановка

In [ ]:
def Q(alpha):
    return 2 * ((2 * delta + 1) * np.cos(alpha) - Bo * U / 2 / pi)

Метод прогонки:

In [ ]:
def getZ(a, b, c, f, n):
    a_i = [0]
    a_i.append(b[0] / c[0])
    b_i = [0]
    b_i.append(f[0] / c[0])
    for i in range(1, n):
        a_i.append(b[i] / (c[i] - a[i] * a_i[i]))
    for i in range(1, n + 1):
        b_i.append((f[i] + a[i] * b_i[i]) / (c[i] - a[i] * a_i[i]))
    z = np.full(n + 1, b_i[-1])
    for i in reversed(range(n)):
        z[i] = a_i[i + 1] * z[i + 1] + b_i[i + 1]
    return z

Разностная схема для свободной поверхности в капилляре

In [ ]:
def Z(alpha, z, h, n):
    kappa1 = 1 / (1 + (1 / 4) * (h ** 2) * Bo)
    nu1 = - (1 / 4) * (h ** 2) * Q(alpha) / (1 + (1 / 4) * (h ** 2) * Bo)
    kappa2 = 1 / (1 + 1 / 2 * h ** 2 / np.sin(alpha) ** 3 * Bo)
    nu2 = (h / np.tan(alpha) - 1 / 2 * h ** 2 / np.sin(alpha) ** 3 * (-np.cos(alpha) / delta + Q(alpha))) * kappa2
    a = [0]
    for i in range(1, n):
        a.append((i - 1 / 2) / (i * h ** 2 * np.sqrt(1 + ((z[i] - z[i - 1]) / h) ** 2)))
    a.append(kappa2)
    
    b = [kappa1]
    for i in range(1, n):
        b.append((i + 1 / 2) / (i * h ** 2 * np.sqrt(1 + ((z[i + 1] - z[i]) / h) ** 2)))

    c = [1]    
    for i in range(1, n):
        s1 = (i + 1 / 2) / (i * h ** 2 * np.sqrt(1 + ((z[i + 1] - z[i]) / h) ** 2))
        s2 = (i - 1 / 2) / (i * h ** 2 * np.sqrt(1 + ((z[i] - z[i - 1]) / h) ** 2))
        c.append(s1 + s2 + Bo)
    c.append(1)
    
    f = [-Q(alpha)] * n
    f[0] = nu1
    f.append(nu2)
    
    z = getZ(a, b, c, f, n)
    return z

Разностная схема для свободной поверхности во внешнем сосуде

In [ ]:
def Z2(alpha, z, h, n):
    kappa1 = 1 / (1 + 1 / 2 * h ** 2 / np.sin(alpha) ** 3 * Bo)
    nu1 = (h / np.tan(alpha) - 1 / 2 * h ** 2 / np.sin(alpha) ** 3 * (np.cos(alpha) / delta + Q(alpha))) * kappa1
    kappa2 = kappa1
    nu2 = (h / np.tan(alpha) - 1 / 2 * h ** 2 / np.sin(alpha) ** 3 * (-np.cos(alpha) + Q(alpha))) * kappa1
    a = [0]
    for i in range(1, n):
        a.append((delta + h * (i - 1 / 2)) / ((delta + i * h) * h ** 2 * np.sqrt(1 + ((z[i] - z[i - 1]) / h) ** 2)))
    a.append(kappa2)
    
    b = [kappa1]
    for i in range(1, n):
        b.append((delta + h * (i + 1 / 2)) / ((delta + i * h) * h ** 2 * np.sqrt(1 + ((z[i + 1] - z[i]) / h) ** 2)))

    c = [1]    
    for i in range(1, n):
        s1 = (delta + h * (i + 1 / 2)) / ((delta + i * h) * h ** 2 * np.sqrt(1 + ((z[i + 1] - z[i]) / h) ** 2))
        s2 = (delta + h * (i - 1 / 2)) / ((delta + i * h) * h ** 2 * np.sqrt(1 + ((z[i] - z[i - 1]) / h) ** 2))
        c.append(s1 + s2 + Bo)
    c.append(1)
    
    f = [-Q(alpha)] * n
    f[0] = nu1
    f.append(nu2)
    
    z = getZ(a, b, c, f, n)
    return z

In [ ]:
R1 = 0.002
R2 = 0.05
delta = R1 / R2
Bo = 300
n = 100
U = 3
pi = np.pi
h = delta / n
z0 = U / pi
gamma = np.linspace(pi / 2, pi / 4, 10)
z = np.full(n + 1, z0)
zk = np.full(n + 1, z0)
q = 0.1

In [ ]:
for i in gamma:    
    z = Z(i, zk, h, n)
    z_k = z[:] * q + zk[:] * (1 - q)
    while max(abs(z - z_k)) / q >= 1e-06:
        zk = z_k
        z = Z(i, zk, h, n)
        z_k = z[:] * q + zk[:] * (1 - q)
    zk = z

In [ ]:
n = int((1 - delta) // h + 1)
z2 = np.full(n + 1, U / pi)
zk2 = np.full(n + 1, U / pi)

In [ ]:
for i in gamma:    
    z2 = Z2(i, zk2, h, n)
    z_k2 = z2[:] * q + zk2[:] * (1 - q)
    while max(abs(z2 - z_k2)) / q >= 1.0e-6:
        zk2 = z_k2
        z2 = Z2(i, zk2, h, n)
    z_k2 = z2[:] * q + zk2[:] * (1 - q)
    zk2 = z2

# Параметрическая постановка

In [ ]:
def Q(delta, alpha, Bo, U):
    return 2 * ((2 * delta + 1) * math.cos(alpha) - Bo * U / 2 / pi)

Метод прогонки:

In [ ]:
def getZ(a, b, c, f, n):
    a_i = [0]
    a_i.append(b[0] / c[0])
    b_i = [0]
    b_i.append(f[0] / c[0])
    for i in range(1, n - 1):
        a_i.append(b[i] / (c[i] - a[i] * a_i[i]))
    for i in range(1, n):
        b_i.append((f[i] + a[i] * b_i[i]) / (c[i] - a[i] * a_i[i]))
    z = np.full(n, b_i[-1])
    for i in reversed(range(n - 1)):
        z[i] = a_i[i + 1] * z[i + 1] + b_i[i + 1]
    return z

In [ ]:
def Z(alpha, z, r, h, n, delta, Bo, U, L1):
    a = np.full(n, 1 / h ** 2)
    b = np.full(n, 1 / h ** 2)
    
    kappa1 = 1 / (1 + 0.25 * h ** 2 * Bo * L1 ** 2)
    nu1 = -0.25 * h ** 2 * L1 * Q(delta, alpha, Bo, U) / (1 + 0.25 * h ** 2 * Bo * L1 ** 2)
    kappa2 = 1 / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L1 ** 2)
    nu2 = (h * math.cos(alpha) - 0.5 * h ** 2 * math.sin(alpha) * L1 * (-math.cos(alpha) / delta + Q(delta, alpha, Bo, U))) / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L1 ** 2)
    
    a[0] = 0
    a[-1] = kappa2
    b[0] = kappa1
    
    c = 2 / h ** 2 + Bo * L1 ** 2 * (r[2:] - r[:-2]) / 2 / h
    v = [1]
    v.extend(c)
    v.append(1)
    len(v)
    c = v
    
    f = (r[2:] - r[:-2]) / 2 / h * ((z[2:] - z[:-2]) / 2 / h / r[1:-1] - L1 * Q(delta, alpha, Bo, U))
    v = [nu1]
    v.extend(f)
    v.append(nu2)
    len(v)
    f = v
    
    z = getZ(a, b, c, f, n)
    return z    

def R(alpha, z, r, h, n, delta, Bo, U, L1):
    a = np.full(n, 1 / h ** 2)
    b = np.full(n, 1 / h ** 2)
    c = np.full(n, 2 / h ** 2)
    
    kappa1 = 0
    nu1 = 0
    kappa2 = 1
    nu2 = h * math.sin(alpha) + 0.5 * h ** 2 * math.cos(alpha) * (Bo * L1 ** 2 * z[-1] - L1 / delta * math.cos(alpha) + L1 * Q(delta, alpha, Bo, U))
    a[0] = 0
    a[-1] = kappa2
    b[0] = kappa1
    
    c[0] = 1
    c[-1] = 1
    f = []
    for i in range(1, len(z) - 1):
        f.append((z[i + 1] - z[i - 1]) / 2 / h * (Bo * L1 ** 2 * z[i] - (z[i + 1] - z[i - 1]) / 2 / h / r[i] + L1 * Q(delta, alpha, Bo, U)))
    v = [nu1]
    v.extend(f)
    v.append(nu2)
    f = v
    
    r = getZ(a, b, c, f, n)
    return r

In [ ]:
n = 101
h = 1 / (n - 1)
gamma = np.linspace(alpha, 0, 10)
Bo = 300
L1 = delta
z0 = U / pi / L1
z = np.full(n, z0)
zk = np.full(n, z0)
r = np.linspace(0, delta, n) / L1
rk = np.linspace(0, delta, n) / L1
q = 0.1

In [ ]:
for i in gamma:
    z = Z(i, zk, rk, h, n, delta, Bo, U, L1)
    r = R(i, zk, rk, h, n, delta, Bo, U, L1)
    L1 = delta / r[-1]

    while (max(abs(z - zk)) / q >= 1e-06 or max(abs(r - rk)) / q >= 1e-06):        
        zk = z
        rk = r        
        z = Z(i, zk, rk, h, n, delta, Bo, U, L1) * q + zk[:] * (1 - q)
        r = R(i, zk, rk, h, n, delta, Bo, U, L1) * q + rk[:] * (1 - q)
        L1 = delta / r[-1]
    zk = z
    rk = r


In [ ]:
Bo = 300
n = 2401
L2 = 1 - delta
z0 = U / pi / L2
z2 = np.full(n, z0)
zk2 = np.full(n, z0)
r2 = np.linspace(delta, 1, n) / L2
rk2 = np.linspace(delta, 1, n) / L2
gamma = np.linspace(alpha, pi, 100)
h2 = 1 / (n - 1)

In [ ]:
for i in gamma:        
     
    z2 = Z2(i, zk2, rk2, h2, n)
    r2 = R2(i, z2, rk2, h2, n)
    L2 = 1 / r2[-1] 
    while max(abs(z2 - zk2)) / q > 1e-06  or max(abs(r2 - rk2)) / q >= 1e-06:
        rk2 = r2
        zk2 = z2
        z2 = Z2(i, zk2, rk2, h2, n)* q + zk2[:] * (1 - q)
        r2 = R2(i, zk2, rk2, h2, n) * q + rk2[:] * (1 - q)
        L2 = 1 / r2[-1]
    rk2 = r2
    zk2 = z2

Другой алгоритм:

In [ ]:
def Z(alpha, z, r, h, n, delta, Bo, U, L1):
    a = np.full(n, 1 / h ** 2)
    b = np.full(n, 1 / h ** 2)
    
    kappa1 = 1 / (1 + 0.25 * h ** 2 * Bo * L1 ** 2)
    nu1 = -0.25 * h ** 2 * L1 * Q(delta, alpha, Bo, U) / (1 + 0.25 * h ** 2 * Bo * L1 ** 2)
    kappa2 = 1 / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L1 ** 2)
    nu2 = (h * math.cos(alpha) - 0.5 * h ** 2 * math.sin(alpha) * L1 * (-math.cos(alpha) / delta + Q(delta, alpha, Bo, U))) / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L1 ** 2)
    
    a[0] = 0
    a[-1] = kappa2
    b[0] = kappa1
    
    c = 2 / h ** 2 + Bo * L1 ** 2 * (r[2:] - r[:-2]) / 2 / h
    v = [1]
    v.extend(c)
    v.append(1)
    len(v)
    c = v
    
    f = (r[2:] - r[:-2]) / 2 / h * ((z[2:] - z[:-2]) / 2 / h / r[1:-1] - L1 * Q(delta, alpha, Bo, U))
    v = [nu1]
    v.extend(f)
    v.append(nu2)
    len(v)
    f = v
    
    z = getZ(a, b, c, f, n)
    return z    

def R(alpha, z, r, h, n, delta, Bo, U, L1):
    r[0] = 0
    for i in range(1, n):
        r[i] = r[i - 1] + h * math.sqrt(1 - ((z[i + 1] - z[i - 1]) / h) ** 2)
    return r

In [ ]:
n = 101
h = 1 / (n - 1)
gamma = np.linspace(alpha, 0, 100)
Bo = 300
L1 = delta
z0 = U / pi / L1
z = np.full(n, z0)
zk = np.full(n, z0)
r = np.linspace(0, delta, n) / L1
rk = np.linspace(0, delta, n) / L1
q = 0.1

In [ ]:
for i in gamma:    
    z = Z(i, zk, rk, h, n, delta, Bo, U, L1)
    r = R(i, z, rk, h, n, delta, Bo, U, L1)
    L1 = delta / r[-1]

    while (max(abs(z - zk)) / q > 1e-06 or max(abs(r - rk)) / q > 1e-06):        
        zk = z
        rk = r
        z = Z(i, zk, rk, h, n, delta, Bo, U, L1) * q + zk[:] * (1 - q)
        r = R(i, zk, rk, h, n, delta, Bo, U, L1) * q + rk[:] * (1 - q)
        L1 = delta / r[-1]        
    zk = z
    rk = r        

In [ ]:
def Z2(alpha, z, r, h, n):
    a = np.full(n, 1 / h ** 2)
    b = np.full(n, 1 / h ** 2)
    
    kappa1 = 1 / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L2 ** 2)
    nu1 = (h * math.cos(alpha) - 0.5 * h ** 2 * math.sin(alpha) * L2 * (math.cos(alpha) / delta + Q(delta, alpha, Bo, U))) / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L2 ** 2)
    kappa2 = 1 / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L2 ** 2)
    nu2 = (h * math.cos(alpha) - 0.5 * h ** 2 * math.sin(alpha) * L2 * (-math.cos(alpha) + Q(delta, alpha, Bo, U))) / (1 + 0.5 * h ** 2 * math.sin(alpha) * Bo * L2 ** 2)
    
    a[0] = 0
    a[-1] = kappa2
    b[0] = kappa1
    
    c = 2 / h ** 2 + Bo * L2 ** 2 * (r[2:] - r[:-2]) / 2 / h
    v = [1]
    v.extend(c)
    v.append(1)
    len(v)
    c = v
    
    f = (r[2:] - r[:-2]) / 2 / h * ((z[2:] - z[:-2]) / 2 / h / r[1:-1] - L2 * Q(delta, alpha, Bo, U))
    v = [nu1]
    v.extend(f)
    v.append(nu2)
    len(v)
    f = v
    
    z = getZ(a, b, c, f, n)
    return z    
def R2(alpha, z, r, h, n):
    r[0] = delta / L2
    for i in range(1, n):
        r[i] = r[i - 1] + h * math.sqrt(1 - ((z[i + 1] - z[i - 1]) / h) ** 2)
    return r

In [ ]:
Bo = 300
n = 2401
L2 = 1 - delta
z0 = U / pi / L2
z2 = np.full(n, z0)
zk2 = np.full(n, z0)
r2 = np.linspace(delta, 1, n) / L2
rk2 = np.linspace(delta, 1, n) / L2
gamma = np.linspace(alpha, 3 * pi / 4, 10)
h2 = 1 / (n - 1)
for i in gamma:
    z2 = Z2(i, zk2, rk2, h2, n)
    r2 = R2(i, z2, rk2, h2, n)
    L2 = 1 / r2[-1] 

    while max(abs(z2 - zk2)) / q > 1e-06  or max(abs(r2 - rk2)) / q >= 1e-06:
        rk2 = r2
        zk2 = z2
        z2 = Z2(i, zk2, rk2, h2, n) * q + zk2[:] * (1 - q)
        r2 = R2(i, zk2, rk2, h2, n) * q + rk2[:] * (1 - q)
        L2 = 1 / r2[-1]
        
    rk2 = r2
    zk2 = z2